In [2]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Input, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

mirrored_strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0","/gpu:1","/gpu:2","/gpu:3","/gpu:4"])

# Создание модели внутри MirroredStrategy
with mirrored_strategy.scope():
    # Загрузка предобученной модели ResNet50
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    
    # Добавление слоев для обучения на задаче сравнения изображений
    x = base_model.output
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    x = Dense(32, activation='relu')(x)
    predictions = Dense(2, activation='softmax')(x)

    # Создание модели
    model = Model(inputs=base_model.input, outputs=predictions)
    
    # Замораживание весов предобученной модели
    for layer in base_model.layers:
        layer.trainable = False

    # Компиляция модели
    model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Подготовка данных для обучения
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        'train',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')

# Обучение модели
model.fit_generator(
        train_generator,
        steps_per_epoch=100,
        epochs=20,
        verbose=1)

# Сохранение модели
model.save('image_similarity_model.h5')


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4')


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4')


Found 369 images belonging to 371 classes.


/tmp/ipykernel_3616/1874537409.py:42: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/20
INFO:tensorflow:batch_all_reduce: 8 all-reduces with algorithm = nccl, num_packs = 1


INFO:tensorflow:batch_all_reduce: 8 all-reduces with algorithm = nccl, num_packs = 1


INFO:tensorflow:batch_all_reduce: 8 all-reduces with algorithm = nccl, num_packs = 1


INFO:tensorflow:batch_all_reduce: 8 all-reduces with algorithm = nccl, num_packs = 1


InvalidArgumentError: Graph execution error:

Detected at node 'replica_4/categorical_crossentropy/softmax_cross_entropy_with_logits' defined at (most recent call last):
    File "/home/user/miniconda3/envs/tf/lib/python3.9/threading.py", line 937, in _bootstrap
      self._bootstrap_inner()
    File "/home/user/miniconda3/envs/tf/lib/python3.9/threading.py", line 980, in _bootstrap_inner
      self.run()
    File "/home/user/miniconda3/envs/tf/lib/python3.9/site-packages/keras/engine/training.py", line 1249, in run_step
      outputs = model.train_step(data)
    File "/home/user/miniconda3/envs/tf/lib/python3.9/site-packages/keras/engine/training.py", line 1051, in train_step
      loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/home/user/miniconda3/envs/tf/lib/python3.9/site-packages/keras/engine/training.py", line 1109, in compute_loss
      return self.compiled_loss(
    File "/home/user/miniconda3/envs/tf/lib/python3.9/site-packages/keras/engine/compile_utils.py",